<a href="https://colab.research.google.com/github/450586509/Knowledge/blob/master/event_triple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install pyltp
import os
from pyltp import Segmentor, Postagger, Parser, NamedEntityRecognizer, SementicRoleLabeller

In [10]:
print("hello")

hello


In [16]:
!wget http://model.scir.yunfutech.com/model/ltp_data_v3.4.0.zip

--2020-04-03 11:50:38--  http://model.scir.yunfutech.com/model/ltp_data_v3.4.0.zip
Resolving model.scir.yunfutech.com (model.scir.yunfutech.com)... 47.246.21.234, 47.246.18.232, 47.246.17.227, ...
Connecting to model.scir.yunfutech.com (model.scir.yunfutech.com)|47.246.21.234|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 637993278 (608M) [application/zip]
Saving to: ‘ltp_data_v3.4.0.zip’

ltp_data_v3.4.0.zip 100%[===================>] 608.44M  10.0MB/s    in 62s     

2020-04-03 11:51:42 (9.86 MB/s) - ‘ltp_data_v3.4.0.zip’ saved [637993278/637993278]



In [18]:
!unzip ltp_data_v3.4.0.zip

Archive:  ltp_data_v3.4.0.zip
   creating: ltp_data_v3.4.0/
  inflating: ltp_data_v3.4.0/cws.model  
  inflating: ltp_data_v3.4.0/md5.txt  
  inflating: ltp_data_v3.4.0/ner.model  
  inflating: ltp_data_v3.4.0/parser.model  
  inflating: ltp_data_v3.4.0/pisrl.model  
  inflating: ltp_data_v3.4.0/pos.model  
  inflating: ltp_data_v3.4.0/version  


In [19]:
!ls

ltp_data_v3.4.0  ltp_data_v3.4.0.zip  sample_data


In [20]:
!ls

ltp_data_v3.4.0  ltp_data_v3.4.0.zip  sample_data


In [0]:
class LtpParser:
    def __init__(self):
        LTP_DIR = "./ltp_data_v3.4.0"
        self.segmentor = Segmentor()
        self.segmentor.load(os.path.join(LTP_DIR, "cws.model"))

        self.postagger = Postagger()
        self.postagger.load(os.path.join(LTP_DIR, "pos.model"))

        self.parser = Parser()
        self.parser.load(os.path.join(LTP_DIR, "parser.model"))

        self.recognizer = NamedEntityRecognizer()
        self.recognizer.load(os.path.join(LTP_DIR, "ner.model"))

        self.labeller = SementicRoleLabeller()
        self.labeller.load(os.path.join(LTP_DIR, 'pisrl.model'))

    '''语义角色标注'''
    def format_labelrole(self, words, postags):
        arcs = self.parser.parse(words, postags)
        roles = self.labeller.label(words, postags, arcs)
        roles_dict = {}
        for role in roles:
            roles_dict[role.index] = {arg.name:[arg.name,arg.range.start, arg.range.end] for arg in role.arguments}
        return roles_dict

    '''句法分析---为句子中的每个词语维护一个保存句法依存儿子节点的字典'''
    def build_parse_child_dict(self, words, postags, arcs):
        child_dict_list = []
        format_parse_list = []
        for index in range(len(words)):
            child_dict = dict()
            for arc_index in range(len(arcs)):
                if arcs[arc_index].head == index+1:   #arcs的索引从1开始
                    if arcs[arc_index].relation in child_dict:
                        child_dict[arcs[arc_index].relation].append(arc_index)
                    else:
                        child_dict[arcs[arc_index].relation] = []
                        child_dict[arcs[arc_index].relation].append(arc_index)
            child_dict_list.append(child_dict)
        rely_id = [arc.head for arc in arcs]  # 提取依存父节点id
        relation = [arc.relation for arc in arcs]  # 提取依存关系
        heads = ['Root' if id == 0 else words[id - 1] for id in rely_id]  # 匹配依存父节点词语
        for i in range(len(words)):
            # ['ATT', '李克强', 0, 'nh', '总理', 1, 'n']
            a = [relation[i], words[i], i, postags[i], heads[i], rely_id[i]-1, postags[rely_id[i]-1]]
            format_parse_list.append(a)

        return child_dict_list, format_parse_list

    '''parser主函数'''
    def parser_main(self, sentence):
        words = list(self.segmentor.segment(sentence))
        postags = list(self.postagger.postag(words))
        arcs = self.parser.parse(words, postags)
        child_dict_list, format_parse_list = self.build_parse_child_dict(words, postags, arcs)
        roles_dict = self.format_labelrole(words, postags)
        return words, postags, child_dict_list, roles_dict, format_parse_list

In [0]:
parse = LtpParser()
    sentence = '李克强总理今天来我家了,我感到非常荣幸'
    words, postags, child_dict_list, roles_dict, format_parse_list = parse.parser_main(sentence)
    print(words, len(words))
    print(postags, len(postags))
    print(child_dict_list, len(child_dict_list))
    print(roles_dict)
    print(format_parse_list, len(format_parse_list))